In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
data = pd.read_csv('data/Mental Illness Survey 1.csv')
cols_to_check = ['Unnamed: 28', 'Unnamed: 29']
data.head()

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,I identify as having a mental illness,...,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Age,Gender,Household Income,Region,Device Type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Response,...,Obsessive thinking,Mood swings,Panic attacks,Compulsive behavior,Tiredness,Response,Response,Response,Response,Response
1,6.630447e+09,168522804.0,01/15/2018 03:45:16 AM,01/15/2018 03:48:24 AM,NaN,NaN,NaN,NaN,06f645d7ea5af372d50a62bd17,No,...,Obsessive thinking,NaN,Panic attacks,NaN,NaN,30-44,Male,"$25,000-$49,999",Mountain,Android Phone / Tablet
2,6.630410e+09,168522804.0,01/15/2018 03:17:52 AM,01/15/2018 03:18:57 AM,NaN,NaN,NaN,NaN,abca2776418ff1fe24bb85e21f,Yes,...,NaN,NaN,Panic attacks,NaN,Tiredness,18-29,Male,"$50,000-$74,999",East South Central,MacOS Desktop / Laptop
3,6.630402e+09,168522804.0,01/15/2018 03:10:28 AM,01/15/2018 03:12:49 AM,NaN,NaN,NaN,NaN,3800088cf4e55278b38bbe67f3,No,...,NaN,NaN,NaN,NaN,NaN,30-44,Male,"$150,000-$174,999",Pacific,MacOS Desktop / Laptop
4,6.630335e+09,168522804.0,01/15/2018 02:11:16 AM,01/15/2018 02:12:33 AM,NaN,NaN,NaN,NaN,84585803a3cec189f89fe43d44,No,...,NaN,NaN,NaN,NaN,NaN,30-44,Male,"$25,000-$49,999",New England,Windows Desktop / Laptop


We see that row 1 is not data but a secondary header. The only relevant data it appears to contain is the col names for symptoms (Depression, Anxiety, Obssesive Thoughts...).

So rename those columns.

In [14]:

data.rename(columns={'Unnamed: 28':'Anxiety',
                     'Unnamed: 29':'Depression',
                     'Unnamed: 30':'Obsessive_thinking',
                     'Unnamed: 31':'Mood_swings',
                     'Unnamed: 32':'Panic_attacks',
                     'Unnamed: 33':'Compulsive_behavior',
                     'Unnamed: 34':'Tiredness'
                    }, inplace=True)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 40 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Respondent ID                                                 334 non-null    float64
 1   Collector ID                                                  334 non-null    float64
 2   Start Date                                                    334 non-null    object 
 3   End Date                                                      334 non-null    object 
 4   IP Address                                                    0 non-null      float64
 5   Email Address                                                 0 non-null      float64
 6   First Name                                                    0 non-null      float64
 7   Last Name                                                     0 non-nul

In [16]:
print("Sum of nulls per col:")
for col in data.columns:
    print(f'{col} : {data[col].isnull().sum()}')

Sum of nulls per col:
Respondent ID : 1
Collector ID : 1
Start Date : 1
End Date : 1
IP Address : 335
Email Address : 335
First Name : 335
Last Name : 335
Custom Data 1 : 1
I identify as having a mental illness : 0
Education : 0
I have my own computer separate from a smart phone : 0
I have been hospitalized before for my mental illness : 0
How many days were you hospitalized for your mental illness : 37
I am currently employed at least part-time : 0
I am legally disabled : 0
I have my regular access to the internet : 0
I live with my parents : 0
I have a gap in my resume : 0
Total length of any gaps in my resume in months. : 0
Annual income (including any social welfare programs) in USD : 0
I am unemployed : 0
I read outside of work and school : 0
Annual income from social welfare programs : 0
I receive food stamps : 0
I am on section 8 housing : 0
How many times were you hospitalized for your mental illness : 0
I have one of the following issues in addition to my illness : 283
Anxiety